In [15]:
import random
import os
import numpy as np


# 시드 정하기!!!!!!!

CFG = {
    'SEED':42,
    'NSPLIT' :10

}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

In [16]:
import os
from sklearn.datasets import fetch_covtype
import random
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
import os
import warnings
warnings.filterwarnings("ignore")

In [18]:
import pandas as pd
import numpy as np

train_df = pd.read_csv(r'C:\A\breastCancer\ml/train.csv')
test_df = pd.read_csv(r"C:\A\breastCancer\ml/test.csv")
# CAT_COL fillna
train_df['NG']= train_df['NG'].fillna(train_df['NG'].mode()[0])
train_df['HG']= train_df['HG'].fillna(train_df['HG'].mode()[0])
train_df['HG_score_1']= train_df['HG_score_1'].fillna(train_df['HG_score_1'].mode()[0])
train_df['HG_score_2']= train_df['HG_score_2'].fillna(train_df['HG_score_2'].mode()[0])
train_df['HG_score_3']= train_df['HG_score_3'].fillna(train_df['HG_score_3'].mode()[0])
train_df['DCIS_or_LCIS_type']= train_df['DCIS_or_LCIS_type'].fillna(train_df['DCIS_or_LCIS_type'].mode()[0])
train_df['T_category']= train_df['T_category'].fillna(train_df['T_category'].mode()[0])
train_df['ER']= train_df['ER'].fillna(train_df['ER'].mode()[0])
train_df['PR']= train_df['PR'].fillna(train_df['PR'].mode()[0])
train_df['HER2']= train_df['HER2'].fillna(train_df['HER2'].mode()[0])
train_df['HER2_IHC']= train_df['HER2_IHC'].fillna(train_df['HER2_IHC'].mode()[0])
train_df['HER2_SISH']= train_df['HER2_SISH'].fillna(train_df['HER2_SISH'].mode()[0])
train_df['BRCA_mutation']= train_df['BRCA_mutation'].fillna(train_df['BRCA_mutation'].mode()[0])

# NUL_COL fillna
# median
train_df['size of cancers'] = train_df['size of cancers'].fillna(train_df['size of cancers'].median())
train_df['ER_Allred_score'] = train_df['ER_Allred_score'].fillna(train_df['ER_Allred_score'].median())
train_df['PR_Allred_score'] = train_df['PR_Allred_score'].fillna(train_df['PR_Allred_score'].median())
train_df['KI-67_LI_percent'] = train_df['KI-67_LI_percent'].fillna(train_df['KI-67_LI_percent'].median())
train_df['HER2_SISH_ratio'] = train_df['HER2_SISH_ratio'].fillna(train_df['HER2_SISH_ratio'].median())


target_name = ['N_category']

# 명목형 변수
cat_col_names = [
    # 기본
    #'dx','location of cancers', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','ER','PR','HER2','HER2_IHC','HER2_SISH',"BRCA_mutation"
    # 0 : 5개 제거(location of cancers,ER,PR,HER2_IHC,BRCA_mutation)
    #'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH'
    # 1 : 3개 제거(location of cancers,ER,BRCA_mutation)
    #'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH','PR','HER2_IHC'
    # 2 : 2개 제거((location of cancers,ER)
    'dx', 'number of cancers', 'NG','HG','HG_score_1','HG_score_2','HG_score_3','DCIS_or_LCIS','DCIS_or_LCIS_type','T_category','HER2','HER2_SISH','PR','HER2_IHC','BRCA_mutation'
]

for col_name in cat_col_names:
    cat_values = np.unique(train_df[col_name])
    
    #print(f"{col_name}: {cat_values}\n")


# 수치형 변수
num_col_names = [
    'age','size of cancers','ER_Allred_score','PR_Allred_score','KI-67_LI_percent','HER2_SISH_ratio'
]

feature_columns = (
    num_col_names + cat_col_names + target_name)

train_df = train_df.loc[:,feature_columns]


test = test_df.loc[:,num_col_names + cat_col_names]


X = train_df.loc[:,num_col_names+cat_col_names]
y = train_df.loc[:,target_name]


In [19]:
"""
# split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = SEED)
"""


X = np.array(X)                 #np.array(df.iloc[:, :-1]) # class 열 제외한 feature 열들 모음 -> array 변환
y = np.array(y)                           #df['class']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [20]:
from catboost import CatBoostClassifier


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

clf = CatBoostClassifier(
    max_depth = 2,
    learning_rate=0.113776260350794,
    n_estimators=125,
    random_state=CFG['SEED']) # 모델 선언

clf.fit(X_train,y_train) # 모델 학습
y_pred=clf.predict(X_test) # 예측 라벨
accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

0:	learn: 0.6507980	total: 12.1ms	remaining: 1.5s
1:	learn: 0.6192730	total: 13.5ms	remaining: 828ms
2:	learn: 0.6054064	total: 13.9ms	remaining: 566ms
3:	learn: 0.5919466	total: 14.5ms	remaining: 438ms
4:	learn: 0.5737032	total: 15.1ms	remaining: 362ms
5:	learn: 0.5656151	total: 15.6ms	remaining: 310ms
6:	learn: 0.5550445	total: 16.1ms	remaining: 271ms
7:	learn: 0.5492724	total: 16.5ms	remaining: 241ms
8:	learn: 0.5425246	total: 16.9ms	remaining: 218ms
9:	learn: 0.5339678	total: 17.4ms	remaining: 200ms
10:	learn: 0.5284294	total: 17.9ms	remaining: 185ms
11:	learn: 0.5246709	total: 18.3ms	remaining: 172ms
12:	learn: 0.5189603	total: 18.7ms	remaining: 161ms
13:	learn: 0.5157407	total: 19.2ms	remaining: 152ms
14:	learn: 0.5126332	total: 19.6ms	remaining: 144ms
15:	learn: 0.5104603	total: 20.1ms	remaining: 137ms
16:	learn: 0.5064725	total: 20.5ms	remaining: 130ms
17:	learn: 0.5044486	total: 21.3ms	remaining: 127ms
18:	learn: 0.4997496	total: 22.3ms	remaining: 124ms
19:	learn: 0.4965297	to

In [38]:
# test prediction

test_df_id = test_df.loc[:,'ID']
len(test_df_id),type(test_df_id)
array = np.array(test_df_id.values)
len(array),type(array)
test_pred = clf.predict(test)
len(test_pred),type(test_pred)
predictions = clf.predict_proba(test)
predict = []
for i,k in enumerate(predictions):
    if k[0]<k[1]:
        predict.append(k[1])
    else:
        predict.append(k[0])
test_prob = np.array(predict)
dic_pred={
    'ID' : array,
    'N_category':test_pred,
    'predictions':test_prob
}


In [39]:
df_test_pred = pd.DataFrame(dic_pred)
df_test_pred


,ID,N_category,predictions
0,BC_01_0011,1.0,0.857291
1,BC_01_0220,1.0,0.906581
2,BC_01_0233,0.0,0.876897
3,BC_01_0258,0.0,0.520941
4,BC_01_0260,1.0,0.989474
...,...,...,...
245,BC_01_3328,1.0,0.861596
246,BC_01_3404,0.0,0.803084
247,BC_01_3418,0.0,0.738463
248,BC_01_3438,0.0,0.885019


In [40]:
f1 = np.mean(f1_score_history)
print(f1)
df_test_pred.to_csv(fr'C:\A\breastCancer\ml\output\ml/tabular_3_{f1}.csv')

0.816793893129771


In [6]:
# split 개수, 셔플 여부 및 seed 설정
kf = KFold(n_splits =CFG['NSPLIT'] , shuffle = True, random_state = CFG['SEED'])

# split 개수 스텝 만큼 train, test 데이터셋을 매번 분할
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

## catboost classififier

In [8]:
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy_history = []
f1_score_history = []

# K-fold 검증 과정으로 실제 랜덤 포레스트 모델을 학습하여 정확도 평균을 내는 방법
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = CatBoostClassifier(
      iterations=130,
      max_depth = 2,
      learning_rate=0.113776260350794, 
      loss_function='CrossEntropy',
      random_state=CFG['SEED']) # 모델 선언

    clf.fit(X_train,y_train) # 모델 학습
    y_pred=clf.predict(X_test) # 예측 라벨
    accuracy_history.append(accuracy_score(y_pred, y_test)) # 정확도 측정 및 기록
    f1_score_history.append(f1_score(y_pred, y_test)) # 정확도 측정 및 기록

print("accuracy 각 분할의 정확도 기록 :", accuracy_history)
print("accuracy 평균 정확도 :", np.mean(accuracy_history))

print("f1_score 각 분할의 정확도 기록 :", f1_score_history)
print("f1_score 평균 정확도 :", np.mean(f1_score_history))

0:	learn: 0.6531806	total: 906us	remaining: 117ms
1:	learn: 0.6231843	total: 1.54ms	remaining: 98.7ms
2:	learn: 0.6033487	total: 1.96ms	remaining: 83.2ms
3:	learn: 0.5898321	total: 2.41ms	remaining: 75.9ms
4:	learn: 0.5767786	total: 2.85ms	remaining: 71.2ms
5:	learn: 0.5672109	total: 3.29ms	remaining: 68ms
6:	learn: 0.5569506	total: 4.11ms	remaining: 72.2ms
7:	learn: 0.5516219	total: 4.62ms	remaining: 70.4ms
8:	learn: 0.5429869	total: 5.09ms	remaining: 68.4ms
9:	learn: 0.5365045	total: 5.54ms	remaining: 66.5ms
10:	learn: 0.5299893	total: 5.99ms	remaining: 64.8ms
11:	learn: 0.5258039	total: 6.43ms	remaining: 63.3ms
12:	learn: 0.5204936	total: 6.86ms	remaining: 61.8ms
13:	learn: 0.5161809	total: 7.24ms	remaining: 60ms
14:	learn: 0.5104679	total: 7.66ms	remaining: 58.8ms
15:	learn: 0.5077797	total: 8.11ms	remaining: 57.8ms
16:	learn: 0.5050685	total: 8.5ms	remaining: 56.5ms
17:	learn: 0.5025576	total: 8.97ms	remaining: 55.8ms
18:	learn: 0.4994428	total: 9.4ms	remaining: 54.9ms
19:	learn: 

## test preidiction

In [9]:
test_df_id = test_df.loc[:,'ID']
len(test_df_id),type(test_df_id)
array = np.array(test_df_id.values)
len(array),type(array)
test_pred = clf.predict(test)
len(test_pred),type(test_pred)
dic_pred={
    'ID' : array,
    'N_category':test_pred
}
df_test_pred = pd.DataFrame(dic_pred)
df_test_pred
f1 = np.mean(f1_score_history)
print(f1)
df_test_pred.to_csv(fr'C:\A\breastCancer\ml\output\ml/tabular_3_{f1}.csv')

(250, pandas.core.series.Series)

(250, numpy.ndarray)

(250, numpy.ndarray)

,ID,N_category
0,BC_01_0011,1.0
1,BC_01_0220,1.0
2,BC_01_0233,0.0
3,BC_01_0258,1.0
4,BC_01_0260,1.0
...,...,...
245,BC_01_3328,1.0
246,BC_01_3404,0.0
247,BC_01_3418,0.0
248,BC_01_3438,0.0


0.782332047497151


In [ ]:
"""import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))


# print the scores on training and test set
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
f1score=f1_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
print('LightGBM Model f1score score: {0:0.4f}'.format(f1_score(y_pred, y_test)))"""

In [ ]:
"""from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    loss_function='CrossEntropy'
)
clf.fit(X_train,y_train)
# predict the results
y_pred=clf.predict(X_test)
# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))
y_pred_train = clf.predict(X_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))
# print the scores on training and test set

print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('catboost classififier Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))"""